<center>

# DATA COLLECTION

</center>

## Import libraries

In [17]:
import pandas as pd
import requests
import PyPDF2
import csv

## Import dataset

In [ ]:
url = "https://data.lacity.org/resource/2nrs-mtv8.csv?$select=*&$limit=764061&$offset=0"

response = requests.get(url)
# Verificar si la solicitud fue exitosa (código de estado 200)
if str(response.status_code).startswith('2'):
    # Guardar el contenido en un archivo CSV
    with open("../data/raw/crime_data.csv", "wb") as f:
        f.write(response.content)
else:
    print(f"Error al realizar la solicitud: {response.status_code}")

In [59]:
url_pdf = "https://data.lacity.org/api/views/2nrs-mtv8/files/4591b6bf-5846-4de0-9fb0-8780a77a036c?download=true&filename=MO_CODES_Numerical_20191119.pdf"

# Descargar el archivo PDF
response = requests.get(url_pdf)

# Verificar si la solicitud fue exitosa (código de estado 200)
if str(response.status_code).startswith('2'):
    # Guardar el contenido en un archivo local
    with open("../data/raw/mo_codes_data.pdf", "wb") as pdf_file:
        pdf_file.write(response.content)

    # Abrir el archivo localmente y extraer el texto
    with open("../data/raw/mo_codes_data.pdf", "rb") as local_pdf:
        pdf_reader = PyPDF2.PdfReader(local_pdf)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    # Eliminar la palabra "MO CODES"
    text = text.replace("MO CODES", "")
    # Guardar el texto como un archivo CSV
    with open("../data/raw/mo_codes_data.csv", "w", encoding="utf-8", newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Dividir el texto en líneas y escribir cada línea como una fila en el CSV
        for line in text.split('\n'):
            csv_writer.writerow([line.strip()])

else:
    print(f"Error al realizar la solicitud: {response.status_code}")

In [60]:
pdf = pd.read_csv("../data/raw/mo_codes_data.csv", header=None, names=["Columna Original"])

In [61]:
pdf["mo_code"] = pdf["Columna Original"].str[:4]
pdf["mo_code_desc"] = pdf["Columna Original"].str[4:]

pdf.drop("Columna Original", axis=1, inplace= True)


In [62]:
pdf.head()

,mo_code,mo_code_desc
0,REV:,07/19
1,0100,Suspect Impersonate
2,0101,Aid victim
3,0102,Blind
4,0103,Physically disabled
